In [ ]:
import random
import numpy as np
import tensorflow as tf
from keras.models import load_model, Model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, Callback, EarlyStopping
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Input, Dense, Dropout
from keras import backend as K
from keras import optimizers
from keras.utils import to_categorical
import keras.optimizers
import pydot
import networkx as nx
from IPython.display import SVG
import glob, os
import pandas as pd
from random import shuffle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score, mean_absolute_error, mean_squared_error, accuracy_score
from pycausal import search as s
from pycausal.pycausal import pycausal as pc
from collections import defaultdict
from pycausal import prior as p


#this is a file for shared functions
from causal_assurance import * 

# Select your number of models
num_models = 100

# CAM-10 specified by nbest as percentage here.
nbest = int(num_models * 0.1)

# Select your number of iterations to repeat experiment
num_repeat = 100

# Select your graph size 
nodes = 4

# select your GPU Here
os.environ["CUDA_VISIBLE_DEVICES"]="" #Comment this line out if you want all GPUS (2 hehe)

# python full-display web browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



#pc = pc()
#pc.start_vm(java_max_heap_size = '21000M')
#tetrad = s.tetradrunner()

models = []
model_names = []


# If you want to randomize networks set this to True
randomize = False
if randomize:
    layers = [256, 512, 1024, 2048, 4096]
    for i in range(num_models):
        network = []
        for j in range(3):
            network.append(layers[random.randint(0,len(layers) -1)])
        models.append(network)
        model_names.append('model/sim' + str(i))

else:
    model_layers = [512, 256]
    for i in range(num_models):
        models.append(model_layers)
        model_names.append('models/sim' + str(i))

print(models, model_names)

# This is the main function that will run for the specified number of times

In [ ]:
# for single variables.
bestMSE = []
bestCOMBO = []
bestWRONG = []
worstMSE = []
worstCOMBO = []
worstWRONG = []

# for multiple variables.
m_bestMSE = []
m_bestCOMBO = []
m_bestWRONG = []
m_worstMSE = []
m_worstCOMBO = []
m_worstWRONG = []

averageDegree = []
targetDegree = []
target_inD = []
target_outD = []
descendants = []
graphDiff = []

t = 0
while (t < num_repeat):
    train_size = 10000
    test_mean = 1
    test_var = 2
    test_size = 2000
    
    # w_G is the imposter DAG
    w_G = random_dag(nodes, random.randint(0, nodes * nodes)) # since max number of edges is n^2
    require = []
    for i in w_G.edges:
        require.append([str(i[0]), str(i[1])])  
    w_prior = p.knowledge(requiredirect = require)

    
    G = random_dag(nodes, random.randint(nodes, nodes * nodes)) # since max number of edges is n^2
    df = gen_data2(np.arange(nodes), G.edges, SIZE = train_size)
    require = []
    for i in G.edges:
        require.append([str(i[0]), str(i[1])])  
    prior = p.knowledge(requiredirect = require)
    examine_graph_continuous(df, prior)
    
    
    
    # Check to make sure that graph matches our prior knowledge. Or else abort this test.
    a = set()
    for i in tetrad.getEdges():
        a.add((i[0], i[-1]))
    b = set()
    for i in require:
        b.add((i[0], i[1]))
    print("A = ", a)
    print("B = ", b)
    if a != b:
        continue

    # Need to set our inputs and outputs
    inputs = set(np.arange(nodes))
    target = str([x for x in G.nodes() if G.out_degree(x)==0 and G.in_degree(x)>=1][0])
    inputs.remove(int(target))
    inputs = list(map(str, inputs))
    
    perturb = int(inputs[random.randint(0,nodes - 2)])
    df_test = gen_data2(np.arange(nodes), G.edges, mean = test_mean, var = test_var, SIZE = test_size)
    sdf_test = gen_data1(np.arange(nodes), G.edges, mean = test_mean, var = test_var, SIZE = test_size, perturb = [perturb])
    target = [target]
    
    print("Inputs = ", inputs)
    print("Target = ", target)
    
    x_test = df_test[inputs]
    y_test = df_test[target]
    
    sx_test = sdf_test[inputs]
    sy_test = sdf_test[target]

    causal_split = 0.2
    val_split = 0.2
    train_split = 1 - (causal_split + val_split)

    x_causal = df[inputs][-int(causal_split * len(df)) :]
    y_causal = df[target][-int(causal_split * len(df)) :]

    x_val = df[inputs][int(train_split * len(df)):-int(causal_split * len(df))]
    y_val = df[target][int(train_split * len(df)):-int(causal_split * len(df))]

    x_train = df[inputs][:int(train_split * len(df))]
    y_train = df[target][:int(train_split * len(df))]

    verbosity = 0

    for idx, model_name in enumerate(model_names):
        if idx % 10 == 0:
            print(idx)
        if type(models[idx]) is list:
            #clear session
            keras.backend.clear_session() 
            #get model according to specification
            model = get_model(models[idx], [0.4] * len(models), np.shape(x_train)[1])
            callbacks = [ModelCheckpoint(model_name, verbose= verbosity, monitor='val_loss',save_best_only=True), 
                         EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose= verbosity, mode='auto')]
            model.compile(optimizer = optimizers.SGD(lr = 0.0001, momentum = 0.9, ), loss='mean_squared_error', metrics = ['mse'])
            model.fit(x_train, y_train, epochs = 20, validation_data = (x_val, y_val), callbacks = callbacks, batch_size = 32, verbose = verbosity)
        else:
            models[idx].fit(X,y)



    generalization = []
    metrics = []
    proposed = []
    w_proposed = []
    x_causal.reset_index(drop=True, inplace = True)
    for idx, model_name in enumerate(model_names):
        if type(models[idx]) is list:
            keras.backend.clear_session()
            model = load_model(model_name)
        else:
            model = models[idx]

        y_pred = model.predict(x_test)
        generalization.append(mean_squared_error(y_pred, y_test))

        #### CHECK FOR CAUSAL METRIC HERE
        y_causal_pred = model.predict(x_causal)
        causal_targets = pd.DataFrame(y_causal_pred, columns = target)

        causal_df = x_causal.join(causal_targets)

        metrics.append(mean_squared_error(y_causal_pred, y_causal))

        ll_pred = get_ll_continuous(causal_df, prior)
        proposed.append(ll_pred)
        ll_pred = get_ll_continuous(causal_df, w_prior)
        w_proposed.append(ll_pred)


    good_factor = (1 + len(nx.ancestors(G, int(target[0])))) / len(G.nodes)
    wrong_factor = (1 + len(nx.ancestors(w_G, int(target[0])))) / len(w_G.nodes)
    total = normalize(metrics) + normalize(proposed) * good_factor
    wrong = normalize(metrics) + normalize(w_proposed) * wrong_factor
    final = pd.DataFrame(np.stack((metrics, proposed,  total, wrong, normalize(generalization)), axis = 1), columns = ['metrics', 'proposed', 'combined', 'wrong', 'generalization'])

    bestMSE.append(final.nsmallest(nbest, 'metrics')['generalization'].values)
    bestCOMBO.append(final.nsmallest(nbest, 'combined')['generalization'].values)
    bestWRONG.append(final.nsmallest(nbest, 'wrong')['generalization'].values)

    worstMSE.append(final.nlargest(nbest, 'metrics')['generalization'].values)
    worstCOMBO.append(final.nlargest(nbest, 'combined')['generalization'].values)
    worstWRONG.append(final.nlargest(nbest, 'wrong')['generalization'].values)
    
    generalization = []
    metrics = []
    proposed = []
    w_proposed = []
    for idx, model_name in enumerate(model_names):
        if type(models[idx]) is list:
            keras.backend.clear_session()
            model = load_model(model_name)
        else:
            model = models[idx]

        y_pred = model.predict(sx_test)
        generalization.append(mean_squared_error(y_pred, sy_test))

        #### CHECK FOR CAUSAL METRIC HERE
        y_causal_pred = model.predict(x_causal)
        causal_targets = pd.DataFrame(y_causal_pred, columns = target)
        

        causal_df = x_causal.join(causal_targets)


        metrics.append(mean_squared_error(y_causal_pred, y_causal))

        ll_pred = get_ll_continuous(causal_df, prior)
        proposed.append(ll_pred)
        ll_pred = get_ll_continuous(causal_df, w_prior)
        w_proposed.append(ll_pred)


    total = normalize(metrics) + normalize(proposed) * good_factor
    wrong = normalize(metrics) + normalize(w_proposed) * wrong_factor
    final = pd.DataFrame(np.stack((metrics, proposed,  total, wrong, normalize(generalization)), axis = 1), columns = ['metrics', 'proposed', 'combined', 'wrong', 'generalization'])
    m_bestMSE.append(final.nsmallest(nbest, 'metrics')['generalization'].values)
    m_bestCOMBO.append(final.nsmallest(nbest, 'combined')['generalization'].values)
    m_bestWRONG.append(final.nsmallest(nbest, 'wrong')['generalization'].values)

    m_worstMSE.append(final.nlargest(nbest, 'metrics')['generalization'].values)
    m_worstCOMBO.append(final.nlargest(nbest, 'combined')['generalization'].values)
    m_worstWRONG.append(final.nlargest(nbest, 'wrong')['generalization'].values) 
    
    print("Times = ", t)
    d = []
    for i in G.degree():
        d.append(i[1])
        if str(i[0]) in target:
            targetDegree.append(i[1])
    averageDegree.append(np.mean(d))
    target_inD.append(G.in_degree(int(target[0])))
    target_outD.append(G.out_degree(int(target[0])))
    descendants.append(len(nx.descendants(G, perturb)))
    
    graphDiff.append(nx.difference(G, w_G).edges())

    
    t += 1
    
np.mean(bestMSE), np.mean(bestCOMBO)

In [ ]:
d = dict()
d['bestMSE'] = bestMSE
d['bestCOMBO'] = bestCOMBO
d['bestWRONG'] = bestWRONG 
d['worstMSE'] = worstMSE
d['worstCOMBO'] = worstCOMBO
d['worstWRONG'] = worstWRONG

d['m_bestMSE'] = m_bestMSE
d['m_bestCOMBO'] = m_bestCOMBO
d['m_bestWRONG'] = m_bestWRONG 
d['m_worstMSE'] = m_worstMSE
d['m_worstCOMBO'] = m_worstCOMBO
d['m_worstWRONG'] = m_worstWRONG

d['averageDegree']= averageDegree
d['targetDegree'] = targetDegree
d['target_inD']=  target_inD
d['target_outD'] = target_outD
d['descendants'] = descendants
d['graphDiff']= graphDiff

import pickle

with open(str(nodes) + 'node.pkl', 'wb') as handle:
    pickle.dump(d, handle, protocol=pickle.HIGHEST_PROTOCOL)
